In [1]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
from IPython.display import clear_output
warnings.filterwarnings('ignore')

In [2]:
all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))


     有價證券名稱 市場別    產業別 公開發行/上市(櫃)/發行日
ID                                   
1101     台泥  上市   水泥工業     1962/02/09
1102     亞泥  上市   水泥工業     1962/06/08
1103     嘉泥  上市   水泥工業     1969/11/14
1104     環泥  上市   水泥工業     1971/02/01
1108     幸福  上市   水泥工業     1990/06/06
...     ...  ..    ...            ...
9949     琉園  上櫃  文化創意業     2003/11/21
9950    萬國通  上櫃   塑膠工業     2004/02/17
9951     皇田  上櫃   電機機械     2003/12/18
9960    邁達康  上櫃   運動休閒     2004/12/06
9962     有益  上櫃   鋼鐵工業     2006/07/10

[1796 rows x 4 columns]


## 三種買賣策略

In [3]:
def strategyM11_profit(stock:pd.DataFrame, day, day1M):
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>=day, stock.index<day1M)), 'Adj Close']
    MA5 = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>=day, stock.index<day1M)), '5MA']
    MA10 = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>=day, stock.index<day1M)), '10MA']
    if close_price.iloc[0] - close_price.iloc[1] > 0:
        return 0
    elif abs(MA5[0] - MA10[0])/MA5[0] >=0.05:
        return 0
    else:
        initial_price = 0.7*close_price.iloc[1] + 0.3*close_price.iloc[0]    
    # print(close_price.iloc[0], close_price.iloc[1], initial_price)
    allprofit = 0
    remain_position = 1
    condition12_remain = True
    condition5MA_remain = True
    condition10MA_remain = True
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        conditionstoploss = profit <= -0.05
        condition12 = profit > 0.12
        condition5MA = close_price.iloc[i+1] < stock.loc[stock.index>day, '5MA'].iloc[i+1]
        condition10MA = close_price.iloc[i+1] < stock.loc[stock.index>day, '10MA'].iloc[i+1]
        if conditionstoploss:
            # print('stoploss')
            return profit
        if any([condition12, condition5MA, condition10MA]):
            if condition12 and condition12_remain:
                allprofit += profit*0.3
                condition12_remain = False
                remain_position -= 0.3
                # print('condition12', profit*0.3)
            if all([condition5MA, condition5MA_remain, not condition12_remain]):
                allprofit += profit*0.3
                condition5MA_remain = False
                remain_position -= 0.3
                # print('condition5MA', profit*0.3)
            if all([condition10MA, condition10MA_remain, not condition12_remain]):
                allprofit += profit*0.4
                condition10MA_remain = False
                remain_position -= 0.4
                # print('condition10MA', profit*0.4)
        if remain_position == 0:
            return allprofit
    allprofit += profit*remain_position
    return allprofit
def strategyM21_profit(stock:pd.DataFrame, day, day1M):
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>=day, stock.index<day1M)), 'Adj Close']
    MA5 = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>=day, stock.index<day1M)), '5MA']
    MA10 = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>=day, stock.index<day1M)), '10MA']
    if close_price.iloc[0] - close_price.iloc[1] > 0:
        return 0
    elif abs(MA5[0] - MA10[0])/MA5[0] >=0.05:
        return 0
    else:
        initial_price = 0.7*close_price.iloc[1] + 0.3*close_price.iloc[0]
    allprofit = 0
    remain_position = 1
    condition15_remain = True
    condition10MA_remain = True
    condition20MA_remain = True
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        conditionstoploss = profit <= -0.06
        condition15 = profit > 0.15
        condition10MA = close_price.iloc[i+1] < stock.loc[stock.index>day, '10MA'].iloc[i+1]
        condition20MA = close_price.iloc[i+1] < stock.loc[stock.index>day, '20MA'].iloc[i+1]
        if conditionstoploss:
            # print('stoploss')
            return profit
        if any([condition15, condition10MA, condition20MA]):
            if condition15 and condition15_remain:
                allprofit += profit*0.3
                condition15_remain = False
                remain_position -= 0.3
                # print('condition15', profit*0.3)
            if all([condition10MA, condition10MA_remain, not condition15_remain]):
                allprofit += profit*0.3
                condition10MA_remain = False
                remain_position -= 0.3
                # print('condition10MA', profit*0.3)
            if all([condition20MA, condition20MA_remain, not condition15_remain]):
                allprofit += profit*0.4
                condition20MA_remain = False
                remain_position -= 0.4
                # print('condition20MA', profit*0.4)
        if remain_position == 0:
            return allprofit
    allprofit += profit*remain_position
    return allprofit
def strategyM31_profit(stock:pd.DataFrame, day, day1M):
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>=day, stock.index<day1M)), 'Adj Close']
    MA5 = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>=day, stock.index<day1M)), '5MA']
    MA10 = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>=day, stock.index<day1M)), '10MA']
    if close_price.iloc[0] - close_price.iloc[1] > 0:
        return 0
    elif abs(MA5[0] - MA10[0])/MA5[0] >=0.05:
        return 0
    else:
        initial_price = 0.7*close_price.iloc[1] + 0.3*close_price.iloc[0]
    allprofit = 0
    remain_position = 1
    condition15_remain = True
    condition10MA_remain = True
    condition20MA_remain = True
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        conditionstoploss = profit <= -0.06
        condition15 = profit > 0.15
        condition10MA = close_price.iloc[i+1] < stock.loc[stock.index>day, '10MA'].iloc[i+1]
        condition20MA = close_price.iloc[i+1] < stock.loc[stock.index>day, '20MA'].iloc[i+1]
        if conditionstoploss:
            # print('stoploss')
            return profit
        if any([condition15, condition10MA, condition20MA]):
            if condition15 and condition15_remain:
                allprofit += profit*0.3
                condition15_remain = False
                remain_position -= 0.3
                # print('condition15', profit*0.3)
            if all([condition10MA, condition10MA_remain, not condition15_remain]):
                allprofit += profit*0.2
                condition10MA_remain = False
                remain_position -= 0.2
                # print('condition10MA', profit*0.2)
            if all([condition20MA, condition20MA_remain, not condition15_remain]):
                allprofit += profit*0.5
                condition20MA_remain = False
                remain_position -= 0.5
                # print('condition20MA', profit*0.5)
        if remain_position == 0:
            return allprofit
    allprofit += profit*remain_position
    return allprofit
def strategy9_profit(stock:pd.DataFrame, day, day1M):
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>=day, stock.index<day1M)), 'Adj Close']
    MA5 = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>=day, stock.index<day1M)), '5MA']
    MA10 = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>=day, stock.index<day1M)), '10MA']
    if close_price.iloc[0] - close_price.iloc[1] > 0:
        return 0
    elif abs(MA5[0] - MA10[0])/MA5[0] >=0.05:
        return 0
    else:
        initial_price = 0.7*close_price.iloc[1] + 0.3*close_price.iloc[0]
    profit1 = 0
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        if profit1 != 0:
            if close_price.iloc[i+1] < MA10.iloc[i+1]:
                return (profit+profit1)/2
            elif profit <=0:
                profit = 0
                return (profit+profit1)/2
        else:
            if profit <-0.06:
                profit = -0.06
                break
            elif profit > 0.15:
                profit1 = 0.15
    return profit
# def strategy5_profit(stock, day, day1M):
#     initial_price = stock.loc[stock.index>day, 'Adj Close']
#     close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
#     low_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Low']
#     initial_price = close_price.iloc[0]
#     profit1 = 0
#     last_low = low_price.iloc[0]
#     for i in range(len(close_price)-1):
#         profit = (close_price.iloc[i+1]-initial_price)/initial_price
#         if close_price.iloc[i+1]<last_low:
#             if profit1 != 0:
#                 return (profit1+profit)/2
#             else:
#                 return profit
#         if profit >=  0.17:
#             profit1 = 0.17
#         last_low = low_price.iloc[i]
#     if profit1 != 0:
#         return (profit1+profit)/2
#     else:
#         return profit
def strategy6_profit(stock, day, day1M):
    initial_price = stock.loc[stock.index>day, 'Adj Close']
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    initial_price = close_price.iloc[0]
    last_profit = 0
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        if all([last_profit<0,profit<0]):
            return profit
        if profit <-0.06:
            profit = -0.06
            return profit
        last_profit = profit
    return profit
def strategy7_profit(stock:pd.DataFrame, day, day1M):
    initial_price = stock.loc[stock.index>day, 'Adj Close']
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    initial_price = close_price.iloc[0]
    profit1 = 0
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        if profit1 != 0:
            if profit > 0.32:
                profit = 0.32
                return (profit+profit1)/2
            elif profit <=0:
                profit = 0
                return (profit+profit1)/2
        else:
            if profit <-0.06:
                profit = -0.06
                break
            elif profit > 0.16:
                profit1 = 0.16
    return profit
def strategy8_profit(stock:pd.DataFrame, day, day1M):
    initial_price = stock.loc[stock.index>day, 'Adj Close']
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    initial_price = close_price.iloc[0]
    profit1 = 0
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        if profit1 != 0:
            if profit > 0.40:
                profit = 0.40
                return (profit+profit1)/2
            elif profit <=0:
                profit = 0
                return (profit+profit1)/2
        else:
            if profit <-0.08:
                profit = -0.08
                break
            elif profit > 0.20:
                profit1 = 0.20
    return profit
def test_strategy():
    d = -39
    day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
    day1M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+31))) , '%Y-%m-%d' )
    day2M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+61))) , '%Y-%m-%d' )
    day3M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+92))) , '%Y-%m-%d' )
    print(f'day:{day}, day1M:{day1M}, day2M:{day2M}, day3M:{day3M}')
    stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{"2465"}.TW.csv', parse_dates=['Date'], index_col=['Date'])
    print('strategy9 1M 2M 3M')
    print(strategy9_profit(stock, day, day1M))
    print('--------------------------------')
    print(strategy9_profit(stock, day, day2M))
    print('--------------------------------')
    print(strategy9_profit(stock, day, day3M))
    print('--------------------------------')
    print('strategyM11 1M 2M 3M')
    print(strategyM11_profit(stock, day, day1M))
    print('--------------------------------')
    print(strategyM11_profit(stock, day, day2M))
    print('--------------------------------')
    print(strategyM11_profit(stock, day, day3M))
    print('--------------------------------')
    print('strategyM21 1M 2M 3M')
    print(strategyM21_profit(stock, day, day1M))
    print('--------------------------------')
    print(strategyM21_profit(stock, day, day2M))
    print('--------------------------------')
    print(strategyM21_profit(stock, day, day3M))
    print('--------------------------------')
    print('strategyM31 1M 2M 3M')
    print(strategyM31_profit(stock, day, day1M))
    print('--------------------------------')
    print(strategyM31_profit(stock, day, day2M))
    print('--------------------------------')
    print(strategyM31_profit(stock, day, day3M))
test_strategy()

day:2023-10-13 00:00:00, day1M:2023-11-13 00:00:00, day2M:2023-12-13 00:00:00, day3M:2024-01-13 00:00:00
strategy9 1M 2M 3M
0
--------------------------------
0
--------------------------------
0
--------------------------------
strategyM11 1M 2M 3M
0
--------------------------------
0
--------------------------------
0
--------------------------------
strategyM21 1M 2M 3M
0
--------------------------------
0
--------------------------------
0
--------------------------------
strategyM31 1M 2M 3M
0
--------------------------------
0
--------------------------------
0


In [4]:
print(datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(np.arange(-5109, -2, 7)[0]))) , '%Y-%m-%d' ))
print(datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(np.arange(-5109, -60, 7)[-1]))) , '%Y-%m-%d' ))

2009-11-25 00:00:00
2023-09-20 00:00:00


In [5]:
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5110, -60, 7)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num[:]):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            # print(e)
            pass
    # print(allstock)
    allstock.drop(['RS MA250rate', 'RS MA50rate', 'RS MA20rate'], axis=1, inplace=True)
    allstock.dropna(inplace=True)
    allstock['ID'] = allstock['ID'].astype(int).astype(str)
    allstock.set_index('ID', inplace = True)
    # print(allstock)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            #TM
            allstock.loc[id, 'Name'] = allstock_info.loc[str(id), '有價證券名稱']
            allstock.loc[id, 'business volume 50MA(百萬)'] = round(float(allstock.loc[id, 'Volume 50MA'])*float(allstock.loc[id, 'Adj Close'])/1000000, 3)
            allstock.loc[id, 'Price>20MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '20MA'])
            allstock.loc[id, 'Price>50MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '50MA'])
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            # allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            #T5
            allstock.loc[id, 'RS 20rate>85'] = (allstock.loc[id, 'RS 20rate'] > 85)
            allstock.loc[id, 'RS 250rate>55'] = (allstock.loc[id, 'RS 250rate'] > 55)
            allstock.loc[id, 'RS 250rate<75'] = (allstock.loc[id, 'RS 250rate'] < 75)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            #T5-2
            allstock.loc[id, 'RS EMA20rate>80'] = (allstock.loc[id, 'RS EMA20rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>60'] = (allstock.loc[id, 'RS EMA250rate'] > 60)
            allstock.loc[id, 'RS EMA250rate<80'] = (allstock.loc[id, 'RS EMA250rate'] < 80)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            #, 'year low sort', 'year high sort'
            #T6-
            allstock.loc[id, 'RS EMA250rate>80'] = (allstock.loc[id, 'RS EMA250rate'] > 80)
            allstock.loc[id, 'RS EMA20rate>80'] = (allstock.loc[id, 'RS EMA20rate'] > 80)
            allstock.loc[id, 'RS EMA20rate<99'] = (allstock.loc[id, 'RS EMA20rate'] < 99)
            allstock.loc[id, 'Volume 50MA>250k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            #T11
            allstock.loc[id, 'price>95%50MA'] = ((allstock.loc[id, 'Adj Close']/allstock.loc[id, '50MA'])>0.95)
            allstock.loc[id, 'RS EMA250rate>75'] = (allstock.loc[id, 'RS EMA250rate'] > 75)
            #,'RS 20EMA is 10MAX','RS EMA20rate<99','Volume 50MA>250k'
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.03)
            allstock.loc[id, 'ATR250/price<0.5'] = (allstock.loc[id, 'ATR250/price'] < 0.15)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.03)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.03)            
            #All Template
            allstock.loc[id, 'T5'] = all(allstock.loc[id, ['RS 20rate>85', 'RS 250rate>55', 'RS 250rate<75', 'year low sort', 'year high sort', 'Volume 50MA>100k']])
            allstock.loc[id, 'T5-2'] = all(allstock.loc[id, ['RS EMA20rate>80', 'RS EMA250rate>60', 'RS EMA250rate<80', 'year low sort', 'year high sort', 'Volume 50MA>100k']])
            allstock.loc[id, 'T6'] = all(allstock.loc[id, ['RS EMA250rate>80', 'RS EMA20rate>80', 'RS EMA20rate<99', 'Volume 50MA>250k']])
            allstock.loc[id, 'T11'] = all(allstock.loc[id, ['RS EMA250rate>75','RS 20EMA is 10MAX','RS EMA20rate<99','Volume 50MA>250k','price>95%50MA']])
            allstock.loc[id, 'TM'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            pass
            allstock.drop(id, inplace=True)
            # print(id, '失敗')
        
    ## 篩出本日期符合Template的股票ID
    allstock.dropna(inplace=True)
    T5_ID = allstock.loc[allstock['T5']].index.values.astype(str)
    T6_ID = allstock.loc[allstock['T6'], ['產業別', 'RS EMA250rate']].sort_values(by='RS EMA250rate', ascending=True).iloc[-50:].index.values.astype(str)
    T11_ID = allstock.loc[allstock['T11']].index.values.astype(str)
    TM_ID = allstock.loc[allstock['TM']].index.values.astype(str)
    
    T5_stock_num = len(T5_ID)
    T6_stock_num = len(T6_ID)
    T11_stock_num = len(T11_ID)
    TM_stock_num = len(TM_ID)
    # print('刪除前  T5 股票數量 : ', T5_stock_num, end=' | ')
    # print('T6 股票數量 : ', T6_stock_num, end=' | ')
    # print('T11 股票數量 : ', T11_stock_num, end=' | ')
    # print('TM 股票數量 : ', TM_stock_num, end=' | ')
    all_template_ID = list(set(np.concatenate([T5_ID,T6_ID,T11_ID,TM_ID])))
    # print(all_template_ID)
    apexstock = allstock.loc[all_template_ID, ['產業別', 'T5', 'T6', 'T11', 'TM'
                                            , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                            , 'ATR250/price', 'ATR50/price', 'ATR20/price','RS 250EMA is 10MAX','RS 50EMA is 10MAX','RS 20EMA is 10MAX'
                                            , 'Price>150MA', 'Price>200MA','year high sort', 'year low sort'
                                            , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']]
    # 防止用濾網篩選前50個的template混亂
    apexstock[['T5','T6','T11','TM']] = False
    apexstock.loc[T5_ID,'T5'] = True
    apexstock.loc[T6_ID,'T6'] = True
    apexstock.loc[T11_ID,'T11'] = True
    apexstock.loc[TM_ID,'TM'] = True
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    column_name = ['產業別', 'T5', 'T6', 'T11', 'TM'
                , 'S250rate', 'S50rate', 'S20rate', 'ES250rate', 'ES50rate', 'ES20rate'
                , 'ATR250/price', 'ATR50/price', 'ATR20/price','ES250 is 10D MAX','ES50 is 10D MAX','ES20 is 10D MAX'
                , 'Price>150MA', 'Price>200MA','year high sort', 'year low sort'
                , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']

    apexstock.columns = column_name
    apexstock.dropna(inplace=True)
    # print(f'apexstock shape:{apexstock.shape}')
    
    profit_list = []
    IDs = apexstock.index.values
    day1M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+31))) , '%Y-%m-%d' )
    day2M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+61))) , '%Y-%m-%d' )
    day3M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+92))) , '%Y-%m-%d' )
    day6M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+183))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        if ID in all_template_ID:
            # print(ID)
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
            except:
                try:
                    stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
                except Exception as e:
                    # print(e)
                    try:
                        apexstock.drop(ID, inplace=True)
                        # print(f'drop stock : {ID}')
                    except:
                        pass
                    continue
            got_profit_list = ([ID in T5_ID, ID in T6_ID, ID in T11_ID, ID in TM_ID])
            # print(got_profit_list)
            try:
                initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
                oneM_price = stock.loc[stock.index<day1M, 'Adj Close'].iloc[-1]
                close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day6M)), 'Adj Close']
                buy_price = 0.3*close_price.iloc[1] + 0.7*close_price.iloc[0]
                profit1 = round(strategyM11_profit(stock, day, day1M),3)
                profit2 = round(strategyM21_profit(stock, day, day1M),3) 
                profit3 = round(strategy9_profit(stock, day, day1M),3) 
                profit4 = round(strategyM11_profit(stock, day, day6M),3) 
                profit5 = round(strategyM21_profit(stock, day, day6M),3) 
                profit6 = round(strategyM31_profit(stock, day, day6M),3) 
                IDprofit = [ID]
                IDprofit.append(buy_price)
                for temp in got_profit_list:
                    for strategy in [profit1, profit2, profit3, profit4, profit5, profit6]:
                        IDprofit.append(strategy*temp)
                        
                profit_list.append(IDprofit)
                # profit_list.append([ID, got_profit_list[0]*profit1, got_profit_list[0]*profit2, got_profit_list[0]*profit3, got_profit_list[0]*profit4
                #                     , got_profit_list[1]*profit1, got_profit_list[1]*profit2, got_profit_list[1]*profit3, got_profit_list[1]*profit4
                #                     , got_profit_list[2]*profit1, got_profit_list[2]*profit2, got_profit_list[2]*profit3, got_profit_list[2]*profit4
                #                     , got_profit_list[3]*profit1, got_profit_list[3]*profit2, got_profit_list[3]*profit3, got_profit_list[3]*profit4])
            except Exception as e:
                # print(e)
                try:
                    apexstock.drop(ID, inplace=True)
                    # print(f'drop stock : {ID}')
                except:
                    pass
                continue
        else:
            pass
    T5_stock_num = len(apexstock.loc[apexstock['T5']].index.values)
    T6_stock_num = len(apexstock.loc[apexstock['T6']].index.values)
    T11_stock_num = len(apexstock.loc[apexstock['T11']].index.values)
    TM_stock_num = len(apexstock.loc[apexstock['TM']].index.values)
    # print('刪除後  T5 股票數量 : ', T5_stock_num, end=' | ')
    # print('T6 股票數量 : ', T6_stock_num, end=' | ')
    # print('T11 股票數量 : ', T11_stock_num, end=' | ')
    # print('TM 股票數量 : ', TM_stock_num, end=' | ')    
    # print(f'apexstock shape : {apexstock.shape}')
    profit_column = ['buy price'
                    ,'T5 M11_1M profit', 'T5 M21_1M profit', 'T5 S9_1M profit', 'T5 M11_6M profit', 'T5 M21_6M profit', 'T5 M31_6M profit'
                    ,'T6 M11_1M profit', 'T6 M21_1M profit', 'T6 S9_1M profit', 'T6 M11_6M profit', 'T6 M21_6M profit', 'T6 M31_6M profit'
                    ,'T11 M11_1M profit', 'T11 M21_1M profit', 'T11 S9_1M profit', 'T11 M11_6M profit', 'T11 M21_6M profit', 'T11 M31_6M profit'
                    ,'TM M11_1M profit', 'TM M21_1M profit', 'TM S9_1M profit', 'TM M11_6M profit', 'TM M21_6M profit', 'TM M31_6M profit']
    profit_df = pd.DataFrame(profit_list, columns=['ID', 'buy price'
                                                   , 'T5 M11_1M profit', 'T5 M21_1M profit', 'T5 S9_1M profit', 'T5 M11_6M profit', 'T5 M21_6M profit', 'T5 M31_6M profit'
                                                   , 'T6 M11_1M profit', 'T6 M21_1M profit', 'T6 S9_1M profit', 'T6 M11_6M profit', 'T6 M21_6M profit', 'T6 M31_6M profit'
                                                   , 'T11 M11_1M profit', 'T11 M21_1M profit', 'T11 S9_1M profit', 'T11 M11_6M profit', 'T11 M21_6M profit', 'T11 M31_6M profit'
                                                   , 'TM M11_1M profit', 'TM M21_1M profit', 'TM S9_1M profit', 'TM M11_6M profit', 'TM M21_6M profit', 'TM M31_6M profit'])
    profit_df.set_index('ID', inplace=True)
    apexstock.loc[apexstock.index.values, profit_column] = profit_df.loc[apexstock.index.values,profit_column]

    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose all/stock choose for all template (Monthstratgy_123M)(fixed and 5-10MA bias)/' + str(day).split(' ')[0] + 'all template選股(Monthstratgy_123M)' + '.xlsx', encoding='utf-8-sig')
    # T5count = len(profit_df.loc[profit_df['T5 M11_1M profit'] != 0, 'T5 M11_1M profit'].loc[apexstock.index.values])
    # if T5count == 0:
    #     T5count = 1
    # T6count = len(profit_df.loc[profit_df['T6 M11_1M profit'] != 0, 'T6 M11_1M profit'].loc[apexstock.index.values])
    # if T6count == 0:
    #     T6count = 1
    # T11count = len(profit_df.loc[profit_df['T11 M11_1M profit'] != 0, 'T11 M11_1M profit'].loc[apexstock.index.values])
    # if T11count == 0:
    #     T11count = 1
    # TMcount = len(profit_df.loc[profit_df['TM M11_1M profit'] != 0, 'TM M11_1M profit'].loc[apexstock.index.values])
    # if TMcount == 0:
    #     TMcount = 1
    profit = {'T5 M11_1M profit':100*profit_df.loc[profit_df['T5 M11_1M profit'] != 0,'T5 M11_1M profit'].mean()
              ,'T5 M21_1M profit':100*profit_df.loc[profit_df['T5 M21_1M profit'] != 0,'T5 M21_1M profit'].mean()
              ,'T5 S9_1M profit':100*profit_df.loc[profit_df['T5 S9_1M profit'] != 0,'T5 S9_1M profit'].mean()
              ,'T5 M11_6M profit':100*profit_df.loc[profit_df['T5 M11_6M profit'] != 0,'T5 M11_6M profit'].mean()
              ,'T5 M21_6M profit':100*profit_df.loc[profit_df['T5 M21_6M profit'] != 0,'T5 M21_6M profit'].mean()
              ,'T5 M31_6M profit':100*profit_df.loc[profit_df['T5 M31_6M profit'] != 0,'T5 M31_6M profit'].mean()
              ,'T5 numbers':T5_stock_num
              ,'T5 buy numbers':len(profit_df.loc[profit_df['T5 M11_1M profit'] != 0,'T5 M11_1M profit'])
              ,'T6 M11_1M profit':100*profit_df.loc[profit_df['T6 M11_1M profit'] != 0,'T6 M11_1M profit'].mean()
              ,'T6 M21_1M profit':100*profit_df.loc[profit_df['T6 M21_1M profit'] != 0,'T6 M21_1M profit'].mean()
              ,'T6 S9_1M profit':100*profit_df.loc[profit_df['T6 S9_1M profit'] != 0,'T6 S9_1M profit'].mean()
              ,'T6 M11_6M profit':100*profit_df.loc[profit_df['T6 M11_6M profit'] != 0,'T6 M11_6M profit'].mean()
              ,'T6 M21_6M profit':100*profit_df.loc[profit_df['T6 M21_6M profit'] != 0,'T6 M21_6M profit'].mean()
              ,'T6 M31_6M profit':100*profit_df.loc[profit_df['T6 M31_6M profit'] != 0,'T6 M31_6M profit'].mean()
              ,'T6 numbers':T6_stock_num
              ,'T6 buy numbers':len(profit_df.loc[profit_df['T6 M11_1M profit'] != 0,'T6 M11_1M profit'])
              ,'T11 M11_1M profit':100*profit_df.loc[profit_df['T11 M11_1M profit'] != 0,'T11 M11_1M profit'].mean()
              ,'T11 M21_1M profit':100*profit_df.loc[profit_df['T11 M21_1M profit'] != 0,'T11 M21_1M profit'].mean()
              ,'T11 S9_1M profit':100*profit_df.loc[profit_df['T11 S9_1M profit'] != 0,'T11 S9_1M profit'].mean()
              ,'T11 M11_6M profit':100*profit_df.loc[profit_df['T11 M11_6M profit'] != 0,'T11 M11_6M profit'].mean()
              ,'T11 M21_6M profit':100*profit_df.loc[profit_df['T11 M21_6M profit'] != 0,'T11 M21_6M profit'].mean()
              ,'T11 M31_6M profit':100*profit_df.loc[profit_df['T11 M31_6M profit'] != 0,'T11 M31_6M profit'].mean()
              ,'T11 numbers':T11_stock_num
              ,'T11 buy numbers':len(profit_df.loc[profit_df['T11 M11_1M profit'] != 0,'T11 M11_1M profit'])
              ,'TM M11_1M profit':100*profit_df.loc[profit_df['TM M11_1M profit'] != 0,'TM M11_1M profit'].mean()
              ,'TM M21_1M profit':100*profit_df.loc[profit_df['TM M21_1M profit'] != 0,'TM M21_1M profit'].mean()
              ,'TM S9_1M profit':100*profit_df.loc[profit_df['TM S9_1M profit'] != 0,'TM S9_1M profit'].mean()
              ,'TM M11_6M profit':100*profit_df.loc[profit_df['TM M11_6M profit'] != 0,'TM M11_6M profit'].mean()
              ,'TM M21_6M profit':100*profit_df.loc[profit_df['TM M21_6M profit'] != 0,'TM M21_6M profit'].mean()
              ,'TM M31_6M profit':100*profit_df.loc[profit_df['TM M31_6M profit'] != 0,'TM M31_6M profit'].mean()
              ,'TM numbers':TM_stock_num
              ,'TM buy numbers':len(profit_df.loc[profit_df['TM M11_1M profit'] != 0,'TM M11_1M profit'])}
    # print(f'T5 M11 count : {len(profit_df.loc[profit_df["T5 M11_1M profit"] != 0, "T5 M11_1M profit"])}\
    #     , T5 count  : {T5_stock_num}\
    #     ,T6 M11 count : {len(profit_df.loc[profit_df["T6 M11_1M profit"] != 0, "T6 M11_1M profit"])}\
    #     , T6 count  : {T6_stock_num}\
    #     ,T11 M11 count : {len(profit_df.loc[profit_df["T11 M11_1M profit"] != 0, "T11 M11_1M profit"])}\
    #     , T11 count  : {T11_stock_num}\
    #     , TM M11 count : {len(profit_df.loc[profit_df["TM M11_1M profit"] != 0, "TM M11_1M profit"])}\
    #     , TM count  : {TM_stock_num}')
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        day_profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([day_profit_df, all_df])
        all_df.fillna(0,inplace=True)
    clear_output()
    print('刪除前   T5 股票數量 : ', T5_stock_num, end=' | ')
    print('T6 股票數量 : ', T6_stock_num, end=' | ')
    print('T11 股票數量 : ', T11_stock_num, end=' | ')
    print('TM 股票數量 : ', TM_stock_num)  
    print('刪除後   T5 股票數量 : ', len(profit_df.loc[profit_df['T5 M11_1M profit'] != 0,'T5 M11_1M profit']), end=' | ')
    print('T6 股票數量 : ', len(profit_df.loc[profit_df['T6 M11_1M profit'] != 0,'T6 M11_1M profit']), end=' | ')
    print('T11 股票數量 : ', len(profit_df.loc[profit_df['T11 M11_1M profit'] != 0,'T11 M11_1M profit']), end=' | ')
    print('TM 股票數量 : ', len(profit_df.loc[profit_df['TM M11_1M profit'] != 0,'TM M11_1M profit']), end=' | ')
    print(f'apexstock shape : {apexstock.shape}')
    print('day', day)
    all_df.fillna(0,inplace=True)
    all_df_nonumbers = all_df.drop(['T5 numbers','T6 numbers','T11 numbers','TM numbers'], axis = 1)
    display(all_df_nonumbers)
    df_sum = all_df_nonumbers.sum()
    df_mean = all_df_nonumbers.mean()
    df_sum = pd.concat([df_sum,df_mean], axis=1)
    display(df_sum.transpose())

刪除前   T5 股票數量 :  52 | T6 股票數量 :  47 | T11 股票數量 :  33 | TM 股票數量 :  179
刪除後   T5 股票數量 :  4 | T6 股票數量 :  5 | T11 股票數量 :  4 | TM 股票數量 :  20 | apexstock shape : (243, 49)
day 2023-09-20 00:00:00


,T5 M11_1M profit,T5 M21_1M profit,T5 S9_1M profit,T5 M11_6M profit,T5 M21_6M profit,T5 M31_6M profit,T5 buy numbers,T6 M11_1M profit,T6 M21_1M profit,T6 S9_1M profit,...,T11 M21_6M profit,T11 M31_6M profit,T11 buy numbers,TM M11_1M profit,TM M21_1M profit,TM S9_1M profit,TM M11_6M profit,TM M21_6M profit,TM M31_6M profit,TM buy numbers
2023-09-20,-4.675000,-4.925000,-4.300000,-6.500000,-6.750000,-6.750000,4,-2.700000,-2.680000,-1.340000,...,-7.825000,-7.825000,4,-3.080000,-3.970000,-2.320000,-6.905000,-7.320000,-7.320000,20
2023-09-13,-4.032000,-5.004000,-3.920000,-6.428000,-7.564000,-7.564000,25,-6.142308,-6.907692,-5.203846,...,-7.710526,-7.710526,19,-4.662376,-5.112871,-4.147525,-6.431683,-7.314851,-7.314851,101
2023-09-06,-5.140000,-5.420000,-4.920000,-6.466667,-7.060000,-7.060000,15,-5.631250,-6.400000,-4.787500,...,-7.278571,-7.278571,14,-4.850000,-4.854839,-4.012903,-6.601613,-7.219355,-7.219355,62
2023-08-30,-2.469231,-3.507692,-2.676923,-5.892308,-7.576923,-7.576923,13,-4.115789,-5.557895,-4.178947,...,-7.114286,-7.114286,7,-3.820896,-4.350746,-3.343284,-6.395522,-7.297015,-7.297015,67
2023-08-23,-1.900000,-2.500000,-1.387500,-6.762500,-7.800000,-7.800000,8,-4.800000,-5.435294,-4.111765,...,-7.684615,-7.684615,13,-5.536364,-6.014545,-4.429091,-6.745455,-7.527273,-7.527273,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-12-22,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2.168750,3.931250,2.137500,...,-7.600000,-7.600000,3,6.118182,7.627273,5.127273,-3.145455,-2.681818,-2.336364,11
2009-12-15,-5.500000,-8.100000,-6.000000,-5.500000,-8.100000,-8.100000,1,-2.070000,-2.070000,-1.590000,...,-7.642857,-7.642857,7,-4.025000,-2.975000,-2.725000,-5.950000,-8.750000,-8.750000,4
2009-12-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.194737,-0.194444,-0.044444,...,-4.600000,-4.672727,11,3.054545,2.772727,3.181818,-6.845455,-8.836364,-8.836364,11
2009-12-01,15.000000,12.650000,13.450000,-6.650000,-6.650000,-6.650000,2,10.304762,11.714286,9.847619,...,1.175000,1.525000,12,11.125000,12.225000,11.800000,-7.600000,-7.850000,-7.850000,4


,T5 M11_1M profit,T5 M21_1M profit,T5 S9_1M profit,T5 M11_6M profit,T5 M21_6M profit,T5 M31_6M profit,T5 buy numbers,T6 M11_1M profit,T6 M21_1M profit,T6 S9_1M profit,...,T11 M21_6M profit,T11 M31_6M profit,T11 buy numbers,TM M11_1M profit,TM M21_1M profit,TM S9_1M profit,TM M11_6M profit,TM M21_6M profit,TM M31_6M profit,TM buy numbers
0,851.243764,1050.288841,1847.686558,-706.883241,148.537002,198.343260,3301.000000,828.579590,1247.528895,2170.712219,...,401.059526,478.304930,11107.000000,1124.792722,1428.457424,2273.851021,-820.383754,5.790378,79.500764,22338.000000
1,1.179008,1.454694,2.559123,-0.979063,0.205730,0.274714,4.572022,1.147617,1.727879,3.006527,...,0.555484,0.662472,15.383657,1.557885,1.978473,3.149378,-1.136266,0.008020,0.110112,30.939058


In [6]:
all_df.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\stock choose for all template (Monthstratgy_123M)(fixed and 5-10MA bias) profit.xlsx')

In [ ]:
display(profit_df)
print(type(profit_df.index.values[0]))
print(profit_df.loc['3029'])